# TP 3 – RPG 2D
## Passage du programme initial (INIT) au programme final corrigé (CORRIGÉ)

### Compétences visées
- Mettre en place un **cycle de jeu** cohérent : mise à jour, affichage, événements.
- Implémenter un **système d’attaque** (mêlée + arc) avec gestion de la portée.
- Implémenter le **gain d’XP** et le **changement de niveau**.
- Afficher des **popups** (dégâts, messages) et une **tête de mort** à la mort.
- Gérer des **projectiles** (flèches) visibles et mis à jour à chaque frame.

### Important (adaptations)
- Adapter les constantes dépendant de la carte (dimensions, échelle, calques de collision) à la **map utilisée**.
- Conserver et utiliser la **classe `map.py` du projet** (ne pas la remplacer par une version externe).

Ce TP fournit des copier-coller **issus de la version CORRIGÉE** (référence), à intégrer progressivement dans la version INIT.

## 0) Préparation
1. Décompresser le zip INIT dans un dossier de travail.
2. Ouvrir le projet dans l’IDE.
3. Lancer `main.py` pour vérifier que la fenêtre s’ouvre.

Objectif : disposer d’une base exécutable avant toute modification.

## 1) main.py – Caméra, projectiles, mise à jour des popups
### 1.1 Vérifier la signature de `on_update`
S’assurer que la méthode possède le paramètre `delta_time` :

In [ ]:
    def on_update(self, delta_time):
        # Déplace le joueur, gère les collisions avec les objets de la map
        self.physics_engine.update()

### 1.2 Ajouter la caméra dans `setup()` et la liste `self.projectiles` dans `__init__`
Ajouter les lignes correspondantes (copier-coller depuis CORRIGÉ).

**Action** : repérer l’endroit où `self.projectiles` est initialisé, et où la caméra est créée :

In [ ]:
        self.projectiles = []
        # -- TP 3 / Etape E2 : FIN --


    def setup(self):
        # Couleur de fond de la map.
        arcade.set_background_color(arcade.csscolor.CORNFLOWER_BLUE)
        
        # Création de la caméra
        self.camera = arcade.Camera2D()
        self.camera.match_window()

        
        # Création de la map
        self.map = Map(self)

### 1.3 Mettre à jour les projectiles et les popups dans `on_update`
Insérer le bloc suivant dans `on_update` (après la mise à jour du moteur physique et des entités) :

In [ ]:
        for proj in list(self.projectiles):
            proj.update()

        # -- TP3 : mise à jour des textes flottants (dégâts / infos) --
        if self.gui is not None and hasattr(self.gui, "step"):
            self.gui.step(delta_time)
       
        # Positionne la caméra sur le joueur
        self.center_camera_to_player()

### Remarque
Les tests peuvent être réalisés une fois toutes les étapes appliquées (validation en fin de TP).

## 2) gui.py – Popups (dégâts, messages) et dessin
Objectif : afficher des informations **dans le jeu** sans utiliser `arcade.draw_text`.

### 2.1 Remplacer/compléter `Gui.__init__` (attributs popups)
Copier-coller le bloc suivant dans `Gui.__init__` (conserver le reste du constructeur) :

In [ ]:
    def __init__(self, game):
        self.game = game

        # UIManager unique, activé
        if not hasattr(self.game, "manager") or self.game.manager is None:
            self.game.manager = arcade.gui.UIManager()
        self.game.manager.enable()

        self.anchor = None  # UIAnchorLayout
        self.menu_bar = None
        self.player_box = None
        self.clicked_mob_box = None

        # -- TP 2 : Création des textures --
        self.actions_player_textures = []
        self.actions_player_textures_clicked = []
        self.create_textures() 
        self.active = True # Icônes actives ?
        # -- TP 2 : FIN --

        # -- TP 3 / Étape E2 : textes flottants (dégâts / infos) --
        # On garde un système simple (hors UIManager) :
        # chaque popup vit un certain temps et "monte" verticalement.
        # La clé (id(entity), text) permet un anti-spam via "debounce".
        self._popups = []
        self._popup_last_time = {}  # (id(entity), text) -> timestamp
        self._popup_clock = 0.0

### 2.2 Ajouter/Remplacer `show_info`, `show_damage`, `step`
Copier-coller les méthodes suivantes dans la classe `Gui` :

In [ ]:
    def show_info(
        self,
        entity,
        text: str,
        color=arcade.color.WHITE,
        *,
        duration: float = 0.9,
        rise: float = 40,
        size: int = 14,
        debounce: float = 0.3,
        dy: float = 28,
    ):
        """Affiche un petit texte temporaire au-dessus d'une entité.

        Paramètres (compatibles avec constants.show_hit_feedback) :
        - duration : durée d'affichage (secondes)
        - rise     : distance totale de montée verticale (pixels)
        - size     : taille police
        - debounce : anti-spam (même texte sur la même entité)
        - dy       : décalage vertical initial au-dessus de l'entité
        """
        if entity is None:
            return

        # anti-spam : même (entité, texte) si rappelé trop vite
        key = (id(entity), str(text))
        last = self._popup_last_time.get(key, -1e9)
        if self._popup_clock - last < float(debounce):
            return
        self._popup_last_time[key] = self._popup_clock

        text_obj = arcade.Text(
            str(text),
            0,
            0,
            color,
            font_size=int(size),
            anchor_x="center",
            anchor_y="center",
        )

        self._popups.append(
            {
                "entity": entity,
                "text_obj": text_obj,
                "text": str(text),
                "color": color,
                "t": float(duration),
                "t0": float(duration),
                "rise": float(rise),
                "size": int(size),
                "dy": float(dy),
            }
        )

In [ ]:
    def show_damage(self, entity, damage: int):
        """Affiche un nombre de dégâts au-dessus de l'entité."""
        # Couleur par défaut (neutre) : blanc cassé
        self.show_info(entity, str(damage), arcade.color.BONE, duration=0.9, rise=45, size=16, debounce=0.0, dy=34)

In [ ]:
    def step(self, delta_time: float):
        """À appeler à chaque frame pour faire vivre les popups."""
        dt = float(delta_time)
        self._popup_clock += dt

        alive = []
        for p in self._popups:
            p["t"] -= dt
            if p["t"] > 0:
                alive.append(p)
        self._popups = alive

### 2.3 Remplacer `draw()`
Remplacer la méthode `draw()` de `Gui` par :

In [ ]:
    def draw(self):
        self.game.manager.draw()

        # Dessin des popups en coordonnées "monde" (la caméra est encore active)
        for p in self._popups:
            ent = p["entity"]
            if ent is None:
                continue

            # progression 0 -> 1
            t0 = p["t0"] if p["t0"] > 1e-9 else 1.0
            alpha = 1.0 - (p["t"] / t0)
            y_offset = p["dy"] + p["rise"] * alpha

            x = getattr(ent, "center_x", 0)
            y = getattr(ent, "top", getattr(ent, "center_y", 0)) + y_offset

            txt = p.get("text_obj")
            if txt is None:
                continue

            # opacity : 1 -> 0
            remain = max(0.0, min(1.0, p["t"] / t0))
            base = p["color"]
            if len(base) == 3:
                r, g, b = base
            else:
                r, g, b = base[0], base[1], base[2]
            txt.color = (int(r), int(g), int(b), int(255 * remain))

            txt.x = x
            txt.y = y
            txt.text = p["text"]
            txt.draw()

### 2.4 Vérifier `Gui.update()`
Cette méthode met à jour les boîtes (joueur / mob cliqué / etc.). Conserver la signature sans paramètre :

In [ ]:
    def update(self):
        if self.game.clicked_mob is None:
            self.setup()
        else:
            self.setup_clicked_mob(self.game.clicked_mob)

### Remarque
Les tests peuvent être réalisés une fois toutes les étapes appliquées (validation en fin de TP).

## 3) mob.py – Déclencher la mort (animation + tête de mort)
Objectif : centraliser le passage d’un mob à l’état `DEAD`.

### 3.1 Ajouter `die()` dans la classe `Mob` :

In [ ]:
    def die(self):
        """Passe le mob en animation de mort."""
        self.status = DEAD
        self.current_texture_indice = 0
        # Repartir au début du rythme d'animation de mort
        self.current_count_tick_move = 0

### Remarque
Les tests peuvent être réalisés une fois toutes les étapes appliquées (validation en fin de TP).

## 4) player.py – Attaque, dégâts, XP, niveau, Repeat, arc
La majorité des écarts INIT → CORRIGÉ se situe ici.

### 4.1 Remplacer la méthode `attack(self, mob)`
Objectifs :
- Vérifier la cible et la portée.
- Initialiser un combo (`Repeat`).
- Démarrer l’état `ATTACK`.

**Action** : remplacer intégralement `attack` par :

### 4.0 Import à ajouter en haut de `player.py`
Ajouter avec les autres imports, en haut du fichier :

```python
from projectile import Arrow
```

Rôle : permettre la création d’une flèche lors d’une action de type **Bow**.

In [ ]:
    def attack(self, mob):
        """
        TP 3 / Étape E1
        Prépare une attaque (corps-à-corps ou distance) :
        - vérifie la portée
        - lance l'animation
        - initialise le combo
        Les dégâts seront appliqués plus tard dans update().
        """

        # 1) Sécurité : pas de cible
        if mob is None:
            return

        # 2) Sécurité : action sélectionnée
        if self.action_name is None:
            return

        # 3) Récupération de l'action
        action = self.actions[self.action_name]

        # 0) Cooldown global (bloque le lancement d'une nouvelle action)
        if hasattr(self.game, "player_timer") and self.game.player_timer is not None:
            if not self.game.player_timer.is_expired() and not (self.status & ATTACK):
                self.game.gui.show_info(self, "Cooldown", arcade.color.SILVER, debounce=0.5)
                return

        # 4) Calcul distance²
        dx = self.center_x - mob.center_x
        dy = self.center_y - mob.center_y
        dist2 = dx * dx + dy * dy

        # 5) Portée
        rmin = int(action.get("Range_Min", 0))
        rmax = int(action.get("Range_Max", 999999))

        if dist2 < rmin * rmin:
            self.game.gui.show_info(self, "Too close", arcade.color.SILVER, debounce=1.0)
            self.game.gui.active = False
            self.game.gui.update()
            self.game.player_timer = Timer(0.0)
            self.game.player_timer.start()
            self.stop_attack()
            return

        if dist2 > rmax * rmax:
            self.game.gui.show_info(self, "Too far", arcade.color.SILVER, debounce=1.0)
            self.game.gui.active = False
            self.game.gui.update()
            self.game.player_timer = Timer(0.0)
            self.game.player_timer.start()
            self.stop_attack()
            return

        # 6) Lancer animation d'attaque
        if not (self.status & ATTACK):
            # Met le bit ATTACK sans risquer d'alterner on/off si on rappelle attack()
            self.status |= ATTACK
            self.current_texture_indice = 0

        # 7) Initialisation du combo
        self.combo["action"] = action
        self.combo["target"] = mob
        self.combo["hits_left"] = int(action.get("Repeat", 1))
        # Cooldown : on autorise le 1er coup tout de suite
        self.combo["timer"] = Timer(0.0)
        self.combo["timer"].start()


        # 8) Interface bloquée pendant l'action
        self.game.gui.active = True

### Remarque
Les tests peuvent être réalisés une fois toutes les étapes appliquées (validation en fin de TP).

### 4.2 Remplacer la méthode `update(self)`
Objectifs :
- Piloter l’animation d’attaque.
- Appliquer **un coup** en fin d’animation.
- Enchaîner les coups si `Repeat > 1`.
- Gérer l’arc (création d’une flèche).
- Gérer la mort + XP + niveau.

**Action** : remplacer intégralement `update` par :

In [ ]:
    def update(self):
        """
        Update du joueur :
        - TP2 : animation d'attaque et animation de marche
        - TP3 : E2 (exécution du combo pendant l'attaque)
        """

        # =========================================================
        # 1) ANIMATION D'ATTAQUE (TP2) + EXECUTION DU COMBO (TP3 E2)
        # =========================================================
        if self.status & ATTACK:

            # 1.1) Déterminer UNE direction valide (priorité gauche/droite/haut, sinon bas)
            if self.status & WALK_LEFT:
                direction = WALK_LEFT
            elif self.status & WALK_RIGHT:
                direction = WALK_RIGHT
            elif self.status & WALK_UP:
                direction = WALK_UP
            else:
                direction = WALK_DOWN

            # 1.2) Etat d'attaque correspondant dans le dictionnaire textures
            # Exemple : WALK_DOWN + ATTACK = ATTACK_DOWN (si tes constantes sont faites ainsi)
            status_for_texture = direction + ATTACK

            # 1.3) Avancer d'une frame dans l'animation d'attaque
            last_index = len(self.textures[status_for_texture]) - 1
            self.current_texture_indice += 1

            # 1.4) Si on a dépassé la dernière frame et dernier coup : fin attaque => retour marche
            # 3) Fin de l'animation d'attaque
            # --- TP3 / E2 : exécution du combo pendant l'attaque --
            if self.current_texture_indice > last_index:

                # On déclenche un coup EXACTEMENT à la fin de l'animation.
                if self.combo["hits_left"] > 0:
                    action = self.combo["action"]
                    mob = self.combo["target"]

                    # Cible invalide / déjà morte => arrêt immédiat + déblocage IHM
                    if mob is None or mob.attributes["HitPoints"] <= 0:
                        self.game.gui.update()
                        self.stop_attack()
                        return

                    action_type = action.get("Type", "Melee")

                    # --------- Corps-à-corps ---------
                    if action_type != "Bow":
                        damage = int(action.get("Hit", 0))
                        mob.attributes["HitPoints"] -= damage
                        if mob.attributes["HitPoints"] < 0:
                            mob.attributes["HitPoints"] = 0

                        if mob.attributes["HitPoints"] == 0:
                            mob.status = DEAD
                            mob.current_texture_indice = 0
                            if hasattr(mob, "current_count_tick_move"):
                                mob.current_count_tick_move = 0

                            xp_mob = int(mob.attributes.get("Experience", 0))
                            self.attributes["Experience"] += xp_mob
                            self.attributes["Level"] = level_from_xp(self.attributes["Experience"])

                            self.game.clicked_mob = None
                            self.game.gui.update()

                    # --------- Arc (projectile) ---------
                    else:
                        dx = mob.center_x - self.center_x
                        dy = mob.center_y - self.center_y
                        dist = math.hypot(dx, dy) or 1.0
                        vx = (dx / dist) * ARROW_SPEED
                        vy = (dy / dist) * ARROW_SPEED
                        arrow = Arrow(
                            self.game,
                            self.center_x, self.center_y,
                            vx, vy,
                            int(action.get("Hit", 0)),
                            ARROW_SPEED,
                            float(action.get("Range_Max", ARROW_MAX_RANGE)),
                            self,
                            mob,
                            action_snapshot=action,
                        )
                        self.game.projectiles.append(arrow)
                        self.game.sprites_list.append(arrow)

                    # --------- Combo ---------
                    self.combo["hits_left"] -= 1

                    # Si la cible est morte, on termine le combo tout de suite
                    if mob.attributes["HitPoints"] <= 0:
                        self.combo["hits_left"] = 0

                    # Enchaînement immédiat (Repeat > 1) : pas de cooldown ENTRE les coups
                    if self.combo["hits_left"] > 0:
                        self.current_texture_indice = 0
                        self.texture = self.textures[status_for_texture][self.current_texture_indice]
                        return

                    # Fin du combo : cooldown GLOBAL de l'action (bloque l'IHM)
                    cooldown = float(action.get("Cooldown", 0.0))
                    self.game.player_timer = Timer(cooldown)
                    self.game.player_timer.start()

                    # (Le déblocage de l'IHM est géré dans main.on_update)
                    self.stop_attack()
                    return

                # Aucun coup à exécuter : retour à la marche
                self.stop_attack()
                return
            # -- TP3 / E2 : FIN --

            else:
                # 1.5) Attaque en cours => pas de déplacement
                self.change_x = 0
                self.change_y = 0
                self.texture = self.textures[status_for_texture][self.current_texture_indice]

             
                # (TP3) Les dégâts / tirs sont déclenchés en FIN d'animation (bloc ci-dessus).
                # Pendant l'attaque, on ne fait pas l'animation de marche
                return
            # -- TP3 / E2 : FIN --

        # ============================
        # 2) LIMITES DE LA CARTE (TP1)
        # ============================
        if self.center_x < 0:
            self.center_x = 0
        elif self.center_x > MAP_WIDTH * MAP_SCALING:
            self.center_x = MAP_WIDTH * MAP_SCALING

        if self.center_y < 0:
            self.center_y = 0
        elif self.center_y > MAP_HEIGHT * MAP_SCALING:
            self.center_y = MAP_HEIGHT * MAP_SCALING

        # ============================
        # 3) ANIMATION DE MARCHE (TP1)
        # ============================

        # Si le joueur ne bouge pas, pas d'animation
        if self.change_x == 0 and self.change_y == 0:
            return

        # Si changement de direction
        if self.change_x < 0 and self.change_y == 0 and self.status != WALK_LEFT:
            self.status = WALK_LEFT
            self.current_texture_indice = -1
        elif self.change_x > 0 and self.change_y == 0 and self.status != WALK_RIGHT:
            self.status = WALK_RIGHT
            self.current_texture_indice = -1
        elif self.change_y < 0 and self.change_x == 0 and self.status != WALK_DOWN:
            self.status = WALK_DOWN
            self.current_texture_indice = -1
        elif self.change_y > 0 and self.change_x == 0 and self.status != WALK_UP:
            self.status = WALK_UP
            self.current_texture_indice = -1

        # Image suivante
        self.current_texture_indice += 1
        if self.current_texture_indice >= len(self.textures[self.status]):
            self.current_texture_indice = 0

        self.texture = self.textures[self.status][self.current_texture_indice]

### 4.3 Remplacer/ajouter `stop_attack(self)`
Objectif : sortie propre de l’état `ATTACK` et remise à zéro du combo.

**Action** : remplacer intégralement `stop_attack` par :

In [ ]:
    def stop_attack(self):
        """
        Arrête proprement une attaque en cours :
        - enlève le mode ATTACK
        - réinitialise l'action
        - rend le bouton re-cliquable
        - met à jour le GUI
        """
        # 1) Sortie du mode attaque (bitwise)
        if self.status & ATTACK:
            self.status &= ~ATTACK

        # 2) Réinitialisations
        self.action_name = None
        self.current_texture_indice = 0

        # 3) Reset du combo (TP3)
        self.combo["action"] = None
        self.combo["target"] = None
        self.combo["hits_left"] = 0
        self.combo["timer"] = None

        # 4) Réactivation de l'IHM
        self.game.gui.update()

### 4.4 Compléter `__init__` (attributs utilisés par l’attaque)
Certains attributs (combo, timers, etc.) sont attendus par `attack/update/stop_attack`.
Comparer `__init__` INIT vs CORRIGÉ.

**Action** : recopier uniquement les ajouts (attributs manquants) depuis :

In [ ]:
    def __init__(self, file_name, scaling, img_width, img_height, coords, game) :
        # Attention, bien indiquer la taille de l'image qui sera affichée !
        super().__init__(file_name, scaling, img_width, img_height, coords, game)
        
        self.status = 0  

        # Position / image de départ
        self.init_x_pos = 0 
        self.init_y_pos = 0

        # -- TP 2 : Ajout des actions -- 
        self.actions = None     # Dictionnaire des actions
        self.action_name = ""   # Nom de l'action menée en cours (attaque ici)
        # -- TP 2 : FIN --

        # -- TP 3 / Etape E1 ---
        # Action en cours (attaque, tir, sort, etc.)
        self.combo = {
            "action": None,
            "target": None,
            "hits_left": 0,
            "timer": 0.0
        }
        # -- TP 3 / Etape E1 FIN --


        self.game = None
        self.gui = None

### Remarque
Les tests peuvent être réalisés une fois toutes les étapes appliquées (validation en fin de TP).

## 5) Validation finale
Critères de validation :
- Attaquer les zombies (mêlée) : dégâts + popups.
- Gagner de l’XP et changer de niveau.
- Afficher une tête de mort / animation de mort.
- Tirer des flèches visibles et mises à jour.

En cas d’écart, comparer :
- les signatures (`on_update(delta_time)`),
- l’appel à `gui.step(delta_time)` dans `main.py`,
- l’état `ATTACK` et la fin d’animation dans `Player.update()`.

## Correctifs indispensables (projet validé)
### 1) Rafraîchir l’affichage des HP si touché
Dans `projectile.py`, compléter si nécessaire par ce morceau de code ( dans update()).
```python
# --- Appliquer / afficher d'abord les dégâts, puis le tag si présent ---
if dmg > 0:
    tgt.attributes['HitPoints'] -= dmg
    self.game.gui.update()
    self.game.gui.show_damage(tgt, dmg)  # <- le nombre (rouge/orange)
if tag:
    self.game.gui.show_info(
        tgt, tag,
        arcade.color.GOLD if "Crit" in tag else arcade.color.SILVER,
        duration=0.9, rise=40, size=14, debounce=0.0
    )
self.game.gui.show_damage(mob, damage)
```

### 2) Repeat = 2 – Affichage des dégâts à chaque coup
Dans `player.py`, lors d’un coup de mêlée (fin d’animation), afficher le popup **à chaque application** :
```python
mob.attributes['HitPoints'] -= damage
if mob.attributes['HitPoints'] < 0:
    mob.attributes['HitPoints'] = 0
self.game.gui.show_damage(mob, damage)
```

### 3) Flèches – Portée, "Miss" et HP jamais négatifs
Dans `projectile.py` :
- Décrémenter `self.remaining` à chaque sous-pas.
- Si `self.remaining <= 0`, afficher **Miss**, puis supprimer la flèche.
- Après application des dégâts, borner `HitPoints` à 0.
- Si la cible est sélectionnée, appeler `self.game.gui.update()` immédiatement après l’impact.

### 4) Déciblage – Utiliser `game.clicked_mob`
Au moment de la mort d’un mob, le champ de sélection est dans `My_Game` (pas dans `Player`) :
```python
if getattr(self.game, 'clicked_mob', None) is tgt:
    self.game.clicked_mob = None
    self.game.gui.update()
```

## Validation finale
- Mêlée : les zombies perdent des HP, affichage des dégâts en popups.
- Action avec `Repeat = 2` : **2 coups** → **2 retraits de HP** et **2 popups**.
- Mort : passage à `DEAD`, animation + popup **☠**, gain d’XP.
- XP / niveau : l’XP augmente, le niveau change conformément à la table.
- Arc : flèches visibles (SpriteList), déplacement, collision (pas de HP négatifs).
- Longue distance : sans impact → **Miss** ; mur → **Arrow lost!**.